In [1]:
import sys
sys.path.insert(0,"/work/pip")

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable

# In Hosptital Mortality Classification 

- Classifiers For Each Data Input Type
    - Transformer
        - Input: 24 x (17*6) --> 17*6
        - First Use 1D Covultional Attention Layer to determine attention weigts 
        - Classifcation: Attention Weights Calc --> Single Hidden Layer --> Softmax Classifcation Layer 
    - DAE
        - Input: 17*6 
        - Classification: Single Hidden Layer --> Softmax Classifcation Layer 
    - PCA
        - Input: 17*6
        - Classification: Single Hidden Layer --> Softmax Classifcation Layer
    - RAW 
        - Input: 17*24
        - Classification : Two Hidden Layers --> Softmax Classifcation Layer
            - Layer 1: 17*24 --> 17*6 = First Reduce Dimensionality to the same degree

## Tranformer Classifier

In [38]:
class TransformerClassifier(nn.Module):
    def __init__(self, d_model, seq_len, dropout=0.1):
        super(TransformerClassifier, self).__init__()
        self.conv = nn.Conv1d(seq_len, seq_len, d_model)
        self.hidden = nn.Linear(d_model, d_model)
        self.output = nn.Linear(d_model, 2)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        attn_weights = F.softmax(F.relu(self.conv(x)), dim=1).transpose(1,2)
        x = torch.bmm(attn_weights, x)
        x = F.relu(self.dropout(self.hidden(x)))
        x = F.softmax(self.output(x)) 
        return x

In [37]:
#Test Attn_weight convolution method
m = nn.Conv1d(5, 5, 17, stride=1)
input = torch.randn(512, 5, 17)
output = F.softmax(m(input), dim = 1)
output.transpose(1, 2).bmm(input).shape

torch.Size([512, 1, 17])

## DAE/PCA Classifier

In [39]:
class FlatClassifier(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super(FlatClassifier, self).__init__()
        self.hidden = nn.Linear(d_model, d_model)
        self.output = nn.Linear(d_model, 2)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = F.relu(self.dropout(self.hidden(x)))
        x = F.softmax(self.output(x))
        return x

## Raw Classifier

In [40]:
class RawClassifier(nn.Module):
    def __init__(self, d_input, d_model, dropout=0.1):
        super(RawClassifier, self).__init__()
        self.embed = nn.Linear(d_input, d_model)
        self.hidden = nn.Linear(d_model, d_model)
        self.output = nn.Linear(d_model, 2)
        self.embed_dropout = nn.Dropout(dropout)
        self.hidden_dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = F.relu(self.embed_dropout(self.embed(x)))
        x = F.relu(self.hidden_dropout(self.hidden(x)))
        x = F.softmax(self.output(x))
        return x